# Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

**Цели работы:**

-подготовить данные о жилье в Калифорнии в 1990 году для обучения моделей

-обучить линейные модели машинного обучения для получения предсказания

-проанализировать результат предсказания моделей

**Описание данных:**

В колонках датасета содержатся следующие данные:

longitude — широта;

latitude — долгота;

housing_median_age — медианный возраст жителей жилого массива;

total_rooms — общее количество комнат в домах жилого массива;

total_bedrooms — общее количество спален в домах жилого массива;

population — количество человек, которые проживают в жилом массиве;

households — количество домовладений в жилом массиве;

median_income — медианный доход жителей жилого массива;

median_house_value — медианная стоимость дома в жилом массиве(целевой признак);

ocean_proximity — близость к океану

**Краткий план работы:**

-провести предобработку данных: устранить пропуски, дубликаты, кодировать, масштабировать признаки

-обучить модели, получить предсказания

-провести оценку качества моделей используя метрики RMSE, MAE и R2

## Подготовка данных

### Загрузка данных

In [ ]:
import pandas as pd
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression  import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

pyspark_version = pyspark.__version__
if int(pyspark_version[:1]) == 3:
    from pyspark.ml.feature import OneHotEncoder
elif int(pyspark_version[:1]) == 2:
    from pyspark.ml.feature import OneHotEncodeEstimator as OneHotEncoder

import warnings
warnings.filterwarnings('ignore')

Данные представленны в 1 файле, прочитаем его и сохраним

In [ ]:
RANDOM_SEED = 42

spark = SparkSession.builder \
                    .master("local") \
                    .appName("Prediction_of_housing_prices") \
                    .getOrCreate()

df = spark.read.option('header', 'true').csv('/datasets/housing.csv', inferSchema = True)
df.printSchema()
df.show(10)

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR B

In [ ]:
df.count()

20640

В таблице представленны данные: 9 колонок c непрерывным типом данных, 1 колонка с категориальным типом данных представленным строкой, 20640 строк

### Дубликаты, типы данных, пропуски

Проверим наличие дубликатов в таблице, исключая целевой признак:

In [ ]:
df.count() - df.drop('median_house_value').dropDuplicates(df.columns.remove('median_house_value')).count()

0

Явных дубликатов в данных нет.

Приведем типы данных к целочисленным там где это будет логически правильно, обработка и вычисления с типом данных int будет происходить быстрее

In [ ]:
data_columns_int = ['housing_median_age', 'total_rooms', 'total_bedrooms',
                    'population', 'households', 'median_house_value']
for col in data_columns_int:
    df = df.withColumn(col, df[col].cast(IntegerType()))
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: integer (nullable = true)
 |-- total_rooms: integer (nullable = true)
 |-- total_bedrooms: integer (nullable = true)
 |-- population: integer (nullable = true)
 |-- households: integer (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: integer (nullable = true)
 |-- ocean_proximity: string (nullable = true)



Проверим данные на наличие пропусков:

In [ ]:
null_columns = [col_name for col_name in df.columns if df.filter(df[col_name].isNull()).count() > 0]
if null_columns:
    print("Пропуски данных обнаружены в следующих колонках:")
    for col in null_columns:
        print(col + ':')
        print(df.filter(df[col].isNull()).count())
else:
    print('Пропусков в данных не обнаружено')

Пропуски данных обнаружены в следующих колонках:
total_bedrooms:
207


Колонка total_bedrooms-количество спален, зависима от колонки bedrooms_from_rooms-общее количество комнат, примерно в одинаковых долях. Предлагаю заполнить пропуски ср.значением доли (количество спален/общее количество комнат) помноженным на количество спален:

In [ ]:
df_sub = df.withColumn("bedrooms_from_rooms", F.col("total_bedrooms") / F.col("total_rooms"))
df_sub.select('bedrooms_from_rooms').show(10)

+-------------------+
|bedrooms_from_rooms|
+-------------------+
|0.14659090909090908|
|0.15579659106916466|
|0.12951601908657123|
|0.18445839874411302|
| 0.1720958819913952|
|0.23177366702937977|
|0.19289940828402366|
|0.22132731958762886|
| 0.2602739726027397|
| 0.1992110453648915|
+-------------------+
only showing top 10 rows



Получение ср.значения доли:

In [ ]:
mean_bedrooms_from_rooms = df_sub.agg(F.avg("bedrooms_from_rooms")).collect()[0][0]
mean_bedrooms_from_rooms

0.21303883048085015

Заполнение пропусков:

In [ ]:
df = df.withColumn("total_bedrooms",
                   F.when(F.col("total_bedrooms").isNull(),
                          F.col("total_rooms")* mean_bedrooms_from_rooms
                         ).otherwise(F.col("total_bedrooms")))

In [ ]:
df = df.withColumn('total_bedrooms', df['total_bedrooms'].cast(IntegerType()))
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: integer (nullable = true)
 |-- total_rooms: integer (nullable = true)
 |-- total_bedrooms: integer (nullable = true)
 |-- population: integer (nullable = true)
 |-- households: integer (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: integer (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [ ]:
null_columns = [col_name for col_name in df.columns if df.filter(df[col_name].isNull()).count() > 0]
if null_columns:
    print("Пропуски данных обнаружены в следующих колонках:")
    for col in null_columns:
        print(col + ':')
        print(df.filter(df[col].isNull()).count())
else:
    print('Пропусков в данных не обнаружено')

Пропусков в данных не обнаружено


Проверим значения категориального столбца ocean_proximity:

In [ ]:
uni_ocean_proximity = df.select('ocean_proximity').distinct()
uni_ocean_proximity.collect()

[Row(ocean_proximity='ISLAND'),
 Row(ocean_proximity='NEAR OCEAN'),
 Row(ocean_proximity='NEAR BAY'),
 Row(ocean_proximity='<1H OCEAN'),
 Row(ocean_proximity='INLAND')]

### Кодирование

Разделяем df на: выборку для обучения и выборку для тестирования качества модели:

In [ ]:
train_data, test_data = df.randomSplit([.8,.2], seed=RANDOM_SEED)
print(train_data.count(), test_data.count())

16560 4080


Подготовим списки с названием колонок: числовых, текстовых, целевой признак

In [ ]:
categorical_cols = ['ocean_proximity']
numerical_cols  = ['longitude', 'latitude', 'housing_median_age',
                   'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']
target = 'median_house_value'

С помощью трансформера StringIndexer переведем текстовые категории в числовое представление

In [ ]:
indexer = StringIndexer(inputCols=categorical_cols,
                        outputCols=[c+'_idx' for c in categorical_cols])
indexer = indexer.fit(train_data)
train_data = indexer.transform(train_data)
test_data = indexer.transform(test_data)

cols = [c for c in train_data.columns for i in categorical_cols if (c.startswith(i))]
train_data.select(cols).show(3)

+---------------+-------------------+
|ocean_proximity|ocean_proximity_idx|
+---------------+-------------------+
|     NEAR OCEAN|                2.0|
|     NEAR OCEAN|                2.0|
|     NEAR OCEAN|                2.0|
+---------------+-------------------+
only showing top 3 rows



Проверим уникальные значения категорий в числовом представлении:

In [ ]:
uni_ocean_proximity_idx = train_data.select('ocean_proximity_idx').distinct()
uni_ocean_proximity_idx.collect()

[Row(ocean_proximity_idx=0.0),
 Row(ocean_proximity_idx=1.0),
 Row(ocean_proximity_idx=4.0),
 Row(ocean_proximity_idx=3.0),
 Row(ocean_proximity_idx=2.0)]

Cоздадим OHE-кодирование для категорий:

In [ ]:
encoder = OneHotEncoder(inputCols=[c+'_idx' for c in categorical_cols],
                        outputCols=[c+'_ohe' for c in categorical_cols])
encoder = encoder.fit(train_data)
train_data = encoder.transform(train_data)
test_data = encoder.transform(test_data)

cols = [c for c in train_data.columns for i in categorical_cols if (c.startswith(i))]
train_data.select(cols).show(3)

+---------------+-------------------+-------------------+
|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------------+-------------------+-------------------+
|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|
|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|
|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|
+---------------+-------------------+-------------------+
only showing top 3 rows



Собираем категориальные признаки в один вектор:

In [ ]:
categorical_assembler = \
        VectorAssembler(inputCols=[c+'_ohe' for c in categorical_cols],
                                        outputCol='categorical_features')
train_data = categorical_assembler.transform(train_data)
test_data = categorical_assembler.transform(test_data)

cols.append('categorical_features')
train_data.select(cols).show(3)

+---------------+-------------------+-------------------+--------------------+
|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|categorical_features|
+---------------+-------------------+-------------------+--------------------+
|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|       (4,[2],[1.0])|
|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|       (4,[2],[1.0])|
|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|       (4,[2],[1.0])|
+---------------+-------------------+-------------------+--------------------+
only showing top 3 rows



In [ ]:
test_data.select(cols).show(3)

+---------------+-------------------+-------------------+--------------------+
|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|categorical_features|
+---------------+-------------------+-------------------+--------------------+
|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|       (4,[2],[1.0])|
|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|       (4,[2],[1.0])|
|     NEAR OCEAN|                2.0|      (4,[2],[1.0])|       (4,[2],[1.0])|
+---------------+-------------------+-------------------+--------------------+
only showing top 3 rows



In [ ]:
train_data.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: integer (nullable = true)
 |-- total_rooms: integer (nullable = true)
 |-- total_bedrooms: integer (nullable = true)
 |-- population: integer (nullable = true)
 |-- households: integer (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: integer (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 |-- ocean_proximity_idx: double (nullable = false)
 |-- ocean_proximity_ohe: vector (nullable = true)
 |-- categorical_features: vector (nullable = true)



In [ ]:
test_data.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: integer (nullable = true)
 |-- total_rooms: integer (nullable = true)
 |-- total_bedrooms: integer (nullable = true)
 |-- population: integer (nullable = true)
 |-- households: integer (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: integer (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 |-- ocean_proximity_idx: double (nullable = false)
 |-- ocean_proximity_ohe: vector (nullable = true)
 |-- categorical_features: vector (nullable = true)



### Масштабирование

Собираем числовые признаки в один вектор:

In [ ]:
numerical_assembler = VectorAssembler(inputCols=numerical_cols, outputCol='numerical_features')
train_data = numerical_assembler.transform(train_data)
test_data = numerical_assembler.transform(test_data)

train_data.select('numerical_features').show(3)

+--------------------+
|  numerical_features|
+--------------------+
|[-124.35,40.54,52...|
|[-124.3,41.8,19.0...|
|[-124.27,40.69,36...|
+--------------------+
only showing top 3 rows



Шкалируем значения числовых признаков при помощи StandardScaler:

In [ ]:
standardScaler = StandardScaler(inputCol='numerical_features',
                                outputCol='numerical_features_scaled', withStd=True, withMean=True)
standardScaler = standardScaler.fit(train_data)
train_data = standardScaler.transform(train_data)
test_data = standardScaler.transform(test_data)

train_data.select('numerical_features_scaled').show(3)

+-------------------------+
|numerical_features_scaled|
+-------------------------+
|     [-2.3735408774002...|
|     [-2.3486386046674...|
|     [-2.3336972410276...|
+-------------------------+
only showing top 3 rows



Собираем все признаки в один общий вектор:

In [ ]:
all_features = ['categorical_features','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features,
                                  outputCol="features")
train_data = final_assembler.transform(train_data)
test_data = final_assembler.transform(test_data)

train_data.select(all_features).show(3)

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (4,[2],[1.0])|     [-2.3735408774002...|
|       (4,[2],[1.0])|     [-2.3486386046674...|
|       (4,[2],[1.0])|     [-2.3336972410276...|
+--------------------+-------------------------+
only showing top 3 rows



In [ ]:
test_data.select(all_features).show(3)

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (4,[2],[1.0])|     [-2.3486386046674...|
|       (4,[2],[1.0])|     [-2.3137754228413...|
|       (4,[2],[1.0])|     [-2.3137754228413...|
+--------------------+-------------------------+
only showing top 3 rows



In [ ]:
train_data.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: integer (nullable = true)
 |-- total_rooms: integer (nullable = true)
 |-- total_bedrooms: integer (nullable = true)
 |-- population: integer (nullable = true)
 |-- households: integer (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: integer (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 |-- ocean_proximity_idx: double (nullable = false)
 |-- ocean_proximity_ohe: vector (nullable = true)
 |-- categorical_features: vector (nullable = true)
 |-- numerical_features: vector (nullable = true)
 |-- numerical_features_scaled: vector (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
test_data.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: integer (nullable = true)
 |-- total_rooms: integer (nullable = true)
 |-- total_bedrooms: integer (nullable = true)
 |-- population: integer (nullable = true)
 |-- households: integer (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: integer (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 |-- ocean_proximity_idx: double (nullable = false)
 |-- ocean_proximity_ohe: vector (nullable = true)
 |-- categorical_features: vector (nullable = true)
 |-- numerical_features: vector (nullable = true)
 |-- numerical_features_scaled: vector (nullable = true)
 |-- features: vector (nullable = true)



## Обучение моделей

Обучаем модель линейной регрессии на всех имеющихся признаках:

In [ ]:
lr = LinearRegression(labelCol=target, featuresCol='features')

model_all = lr.fit(train_data)

24/03/27 10:33:29 WARN Instrumentation: [add6c05e] regParam is zero, which might cause numerical instability and overfitting.
24/03/27 10:33:29 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/03/27 10:33:29 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
24/03/27 10:33:30 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
24/03/27 10:33:30 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


Получаем предсказания модели, сохраняем результат:

In [ ]:
predictions = model_all.transform(test_data)

predictedLabes_all = predictions.select('median_house_value', 'prediction')
predictedLabes_all = predictedLabes_all.withColumnRenamed('prediction', 'prediction_all')
predictedLabes_all.show()

+------------------+------------------+
|median_house_value|    prediction_all|
+------------------+------------------+
|            103600|150437.85225986573|
|            106700|216920.84023626143|
|             73200|125693.57195801308|
|             90100|194385.29762263136|
|             67000|152547.25736168885|
|             86400|185978.61552679125|
|             70500|162831.02097745007|
|             85100| 179279.1378321476|
|             80500| 181468.9929715983|
|             96000|170106.72256533115|
|             75500|136167.11749753085|
|             75000|104107.44193015184|
|            100600| 189870.8131167517|
|             74100|156296.34770281438|
|             66800|133742.99605191118|
|             72600|161043.85984623432|
|             81100|150349.59826782712|
|            135600|175202.17900883846|
|            119400| 169348.7696372462|
|             71300|170313.46194322186|
+------------------+------------------+
only showing top 20 rows



Обучаем модель lr только на численных признаках:

In [ ]:
lr = LinearRegression(labelCol=target, featuresCol='numerical_features_scaled')

model_num = lr.fit(train_data)

predictions = model_num.transform(test_data)

predictedLabes_num = predictions.select('median_house_value', 'prediction')
predictedLabes_num = predictedLabes_num.withColumnRenamed('prediction', 'prediction_num')
predictedLabes_num.show()

24/03/27 10:33:32 WARN Instrumentation: [1eeca04a] regParam is zero, which might cause numerical instability and overfitting.


+------------------+------------------+
|median_house_value|    prediction_num|
+------------------+------------------+
|            103600|101143.91553244332|
|            106700|190127.39755671666|
|             73200| 75856.95699133343|
|             90100| 161880.7468928225|
|             67000|120019.79137483667|
|             86400|156031.61901644856|
|             70500|130167.07739307357|
|             85100|149814.09987593273|
|             80500|150248.89679873764|
|             96000|133845.40300808396|
|             75500|  98413.7591248618|
|             75000| 50412.99971223538|
|            100600|156486.94942283636|
|             74100|123443.33448965964|
|             66800|102595.77821034718|
|             72600|127571.60610639138|
|             81100|116384.55643442056|
|            135600|152202.68314603053|
|            119400|129974.81201701472|
|             71300|146147.90438503402|
+------------------+------------------+
only showing top 20 rows



Сохраняем все предсказания моделей:

In [ ]:
predictedLabes_num_sub = predictedLabes_num.select('prediction_num').withColumn("index", F.monotonically_increasing_id())
predictedLabes_all_sub = predictedLabes_all.withColumn("index", F.monotonically_increasing_id())

predictedLabes = predictedLabes_all_sub.join(predictedLabes_num_sub, "index", "left").drop("index")
predictedLabes = predictedLabes.select('median_house_value', 'prediction_all', 'prediction_num')
predictedLabes.show()

+------------------+------------------+------------------+
|median_house_value|    prediction_all|    prediction_num|
+------------------+------------------+------------------+
|            103600|150437.85225986573|101143.91553244332|
|            106700|216920.84023626143|190127.39755671666|
|             73200|125693.57195801308| 75856.95699133343|
|             90100|194385.29762263136| 161880.7468928225|
|             67000|152547.25736168885|120019.79137483667|
|             86400|185978.61552679125|156031.61901644856|
|             70500|162831.02097745007|130167.07739307357|
|             85100| 179279.1378321476|149814.09987593273|
|             80500| 181468.9929715983|150248.89679873764|
|             96000|170106.72256533115|133845.40300808396|
|             75500|136167.11749753085|  98413.7591248618|
|             75000|104107.44193015184| 50412.99971223538|
|            100600| 189870.8131167517|156486.94942283636|
|             74100|156296.34770281438|123443.3344896596

In [ ]:
predictedLabes.count()

4080

## Анализ результатов

Оценим качество наших моделей при помощи трансформера RegressionEvaluator:

In [ ]:
RMSE_all = RegressionEvaluator(predictionCol='prediction_all', labelCol='median_house_value', metricName='rmse')
RMSE_all = RMSE_all.evaluate(predictedLabes_all)

MAE_all = RegressionEvaluator(predictionCol='prediction_all', labelCol='median_house_value', metricName='mae')
MAE_all = MAE_all.evaluate(predictedLabes_all)

r2_all = RegressionEvaluator(predictionCol='prediction_all', labelCol='median_house_value', metricName='r2')
r2_all = r2_all.evaluate(predictedLabes_all)

print('Результат предсказания модели обученной на всех признаках:')
print('RMSE: ', RMSE_all)
print('MAE:  ', MAE_all)
print('R2:   ', r2_all)

Результат предсказания модели обученной на всех признаках:
RMSE:  70626.81002393599
MAE:   50787.097486779734
R2:    0.6394770233153882


In [ ]:
RMSE_num = RegressionEvaluator(predictionCol='prediction_num', labelCol='median_house_value', metricName='rmse')
RMSE_num = RMSE_num.evaluate(predictedLabes_num)

MAE_num = RegressionEvaluator(predictionCol='prediction_num', labelCol='median_house_value', metricName='mae')
MAE_num = MAE_num.evaluate(predictedLabes_num)

r2_num = RegressionEvaluator(predictionCol='prediction_num', labelCol='median_house_value', metricName='r2')
r2_num = r2_num.evaluate(predictedLabes_num)

print('Результат предсказания модели обученной только на количественных признаках:')
print('RMSE: ', RMSE_num)
print('MAE:  ', MAE_num)
print('R2:   ', r2_num)

Результат предсказания модели обученной только на количественных признаках:
RMSE:  71592.62134035802
MAE:   51690.736416890155
R2:    0.6295494067818987


Лучшей моделью себя показала lr обученная на всех признаках:

В среднем модель 'промахивается' на 50787, модель показывает результат в 63.9 процентах случаев лучше чем среднее арифметическое константное предсказание

In [ ]:
spark.stop()

## Общий вывод

В проделанной работе есть промежуточные выводы, в общем выводе я бы хотел обобщить их и подвести итог по работе с данными, работа проводилась в рамках SparkSession со Spark DataFrame

<h3>Подготовка данных</h3>

**Загрузка данных**

-Данные представленны в 1 файле, в таблице представленны: 9 колонок c непрерывным типом данных, 1 колонка с категориальным типом данных представленным строкой, 20640 строк

**Дубликаты, типы данных, пропуски**

-Дубликатов в данных не обнаруженно

-Типы данных в колонках:'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_house_value'; приведенны к целочисленным

-Обнаруженны пропуски в 'total_bedrooms'-207 пропусков, заполненны ср.значением доли (количество спален/общее количество комнат) помноженным на количество спален

**Кодирование**

-Данные разделенны на обучающую и тестовую выборку

-С помощью трансформера StringIndexer переведены текстовые категории в числовое представление

-Проведено OHE-кодирование для категорий

-Собраны категориальные признаки в один вектор

**Масштабирование**

-Собраны числовые признаки в один вектор

-Проведено масштабирования значений числовых признаков при помощи StandardScaler

-Собраны все признаки в один общий вектор

<h3>Обучение моделей</h3>

-Обученна модель линейной регрессии на всех имеющихся признаках

-Обученна модель линейной регрессии только на численных признаках

-Результаты предсказания моделей сохраненны в таблицу predictedLabes

<h3>Анализ результатов</h3>

**Полученны метрики качества моделей**

-Результат предсказания модели обученной на всех признаках:

RMSE:  70626.81002393583

MAE:   50787.09748677999

R2:    0.6394770233153899

Результат предсказания модели обученной только на количественных признаках:

RMSE:  71592.62134035784

MAE:   51690.73641689009

R2:    0.6295494067819007

-Лучшей моделью себя показала lr обученная на всех признаках, в среднем модель 'промахивается' на 50787, модель показывает результат в 63.9 процентах случаев лучше чем среднее арифметическое константное предсказание